In [4]:
#pip install csvkit
#pip install jsonlines
import requests  # For sending GET requests from the API
import os  # For saving access tokens and for file management when creating and adding to the dataset
import json # For dealing with json responses we receive from the API
#import jsonlines
import pandas as pd #using pandas dataframe for data handling
import csv # For saving the response data in CSV format
import datetime # For parsing the dates received from twitter in readable formats
import dateutil.parser
import unicodedata
import time #To add wait time between requests
from time import gmtime, strftime, localtime # For current time

from config.pathDefinitions import ROOT_DIR #for handling relative paths
from dataAppendFormats.appendCSV import append_to_csv
from dataAppendFormats.appendJSON import append_to_json
from main_program import endpoint2_main

APIpath = os.path.join(ROOT_DIR, 'APICredentials.json')
APICredentials = json.loads(open(APIpath).read())  
bearer_token = APICredentials['bearer_token']
endpoint = APICredentials[input("Enter endpoint name: ")]
print("endpointURL_In_Use: ""{}".format(endpoint))  #tweet count/volume
print("APICredentials: " "{}".format(APIpath))

def create_headers(r):
    """
    Method required by bearer token authentication.
    """
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r

#def bearer_oauth(r):
    #"""
    #Method required by bearer token authentication.
    #"""
    #r.headers["Authorization"] = f"Bearer {bearer_token}"
    #r.headers["User-Agent"] = "v2UserLookupPython"
    #return r

#def create_url(keyword, start_date, end_date, max_results = 10):
def create_url(max_results = 10):
    
    
    #usernames="usernames=TwitterDev,TwitterAPI"   
    #user_fields = "user.fields=created_at,description,public_metrics,verified"
    ids="ids=2392531752,6253282,1696874269"
    user_fields = "user.fields=id,name,username,created_at,description,public_metrics,verified"
    
    url = endpoint.format(ids, user_fields) 
    #url = endpointURL.format(usernames, user_fields)

    #change params based on the endpoint you are using
    #query_params = {#'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    #'max_results': max_results,
                    #'usernames': usernames,
                    #'expansions': 'author_id,in_reply_to_user_id,geo.place_id', 
                    #'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,entities,referenced_tweets,reply_settings,source',
                    #'user.fields': 'id,name,username,created_at,description,public_metrics,verified'
                    #'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    #'next_token': {}
                   #}
    #return (url, query_params)
    return (url)

def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=create_headers,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #for user in json_response['data']:
        #follower_count = user['public_metrics']
        #print(follower_count)
      
    
    #Loop through each 
    for user in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        try:
            author_id = user['id']
        except KeyError: #else:
            author_id = " "
             
        # 2 Name
        try:   
            name = user['name']
        except KeyError:
            name = " "
        
       
        # 3. Time created
        try: #if ('created_at' in tweet):
            created_at = dateutil.parser.parse(user['created_at'])
        except: #else: 
            created_at = " "
            
        # 4. Description
        try:   
            description = user['description']
        except KeyError:
            description = " "
        
        # 5. verified
        try:   
            verified = user['verified']
        except KeyError:
            verified = " "       
        
        # 6
        if('public_metrics' in user) and 'followers_count' in (user['public_metrics']):
            followers_count = user['public_metrics']['followers_count']
        else: followers_count = " "

        # 7
        if('public_metrics' in user) and 'following_count' in (user['public_metrics']):
            following_count = user['public_metrics']['following_count']
        else: following_count = " "
        
        # 8
        if('public_metrics' in user) and 'tweet_count' in (user['public_metrics']):
            tweet_count = user['public_metrics']['tweet_count']
        else: tweet_count = " "
            
        # 9
        if('public_metrics' in user) and 'listed_count' in (user['public_metrics']):
            listed_count = user['public_metrics']['listed_count']
        else: listed_count = " "


        # Assemble all data in a list
        #res = [author_id, created_at, geo, tweet_id, lang, like_count, organic_like_count, quote_count, reply_count, organic_reply_count, retweet_count, organic_retweet_count, source, text, conversation_id]
        res = [author_id, name, created_at, description, verified, followers_count, following_count, tweet_count, listed_count]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1
        
    # When done, close the CSV file
    csvFile.close()  
    
    #Print the number of tweets for this iteration
    print("# of users added from this response: ", counter)
    
def endpointURL4main():
    
    total_users = 0 #counting users
    DataPath = "userDataExploring2022.csv" 
    
    url = create_url()
    json_response = connect_to_endpoint(url)
    print(json.dumps(json_response, indent=4, sort_keys=True))
        
    # Create file
    csvFile = open(DataPath, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Create headers for the data you want to save, in this example, we only want save these columns in our dataset
    csvWriter.writerow(['author_id', 'name', 'created_at', 'description', 'verified', 'followers_count', 'following_count', 'tweet_count', 'listed_count'])
    append_to_csv(json_response, DataPath)
    csvFile.close()
     
if __name__ == "__main__":
    endpointURL4main()

Enter endpoint name: endpointURL4b
endpointURL_In_Use: https://api.twitter.com/2/users?{}&{}
APICredentials: C:\Users\niina\OneDrive - The University of Texas-Rio Grande Valley\Twitter Data Collection Workshop Folder\TwitterDataCollectionWorkshop\APICredentials.json
200
{
    "data": [
        {
            "created_at": "2014-03-16T11:02:50.000Z",
            "description": "Author of cryptocrrency millionaire. The ultimate cryptocurrency trading, strategy bible.",
            "id": "2392531752",
            "name": "Jason Darkcoin Bear",
            "public_metrics": {
                "followers_count": 3463,
                "following_count": 1934,
                "listed_count": 22,
                "tweet_count": 232
            },
            "username": "cryptomillion",
            "verified": false
        },
        {
            "created_at": "2007-05-23T06:01:13.000Z",
            "description": "Tweets about changes and service issues. Follow @TwitterDev\u00a0for more.",
   

In [8]:
import pandas as pd
df1 = pd.read_csv('userdataExploring2022.csv')
df1

,author_id,name,created_at,description,verified,followers_count,following_count,tweet_count,listed_count
0,author_id,name,created_at,description,verified,followers_count,following_count,tweet_count,listed_count
1,author_id,name,created_at,description,verified,followers_count,following_count,tweet_count,listed_count
2,author_id,name,created_at,description,verified,followers_count,following_count,tweet_count,listed_count
3,author_id,name,created_at,description,verified,followers_count,following_count,tweet_count,listed_count
4,author_id,name,created_at,description,verified,followers_count,following_count,tweet_count,listed_count
5,author_id,name,created_at,description,verified,followers_count,following_count,tweet_count,listed_count
6,author_id,name,created_at,description,verified,followers_count,following_count,tweet_count,listed_count
7,2392531752,Jason Darkcoin Bear,2014-03-16 11:02:50+00:00,Author of cryptocrrency millionaire. The ultim...,False,3463,1934,232,22
8,6253282,Twitter API,2007-05-23 06:01:13+00:00,Tweets about changes and service issues. Follo...,True,6043308,38,3729,12311
9,author_id,name,created_at,description,verified,followers_count,following_count,tweet_count,listed_count
